In [ ]:
# 🎯 Age Detection with Fine-Tuned CNN

This notebook implements age detection using a fine-tuned pre-trained CNN model on the UTK Face dataset.

## 📋 Features:
- Pre-trained CNN models (ResNet50, ResNet34, EfficientNet-B0)
- Two-phase fine-tuning strategy
- Comprehensive evaluation metrics
- Real-time training visualization

## 🚀 Expected Performance:
- MAE: 4-6 years
- Accuracy (±5 years): 65-75%
- Accuracy (±10 years): 85-90%

In [ ]:
# Install required packages
!pip install torch torchvision
!pip install scikit-learn matplotlib tqdm opencv-python

# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import files
import zipfile
import os

# Create dataset directory
os.makedirs('/content/UTKFace_data', exist_ok=True)

print("Please upload your UTK Face dataset archive (zip file)")
print("From your path: C:\\Users\\2425605\\Downloads\\archive")
print("If it's a zip file, upload it. If it's a folder, zip it first then upload.")

uploaded = files.upload()

# Extract if it's a zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/UTKFace_data')
        print("Dataset extracted successfully!")

# Check dataset structure
print("\nDataset structure:")
for root, dirs, files in os.walk('/content/UTKFace_data'):
    level = root.replace('/content/UTKFace_data', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files)-5} more files")

Please upload your UTK Face dataset archive (zip file)
From your path: C:\Users\2425605\Downloads\archive
If it's a zip file, upload it. If it's a folder, zip it first then upload.


Saving archive.zip to archive (1).zip
Extracting archive (1).zip...
Dataset extracted successfully!

Dataset structure:
UTKFace_data/
  UTKFace/
    1_0_3_20161220215943341.jpg.chip.jpg
    24_1_4_20170117194842219.jpg.chip.jpg
    28_0_1_20170113151917402.jpg.chip.jpg
    36_0_0_20170117181851709.jpg.chip.jpg
    27_1_0_20170117142744825.jpg.chip.jpg
    ... and 23703 more files
  utkface_aligned_cropped/
    UTKFace/
      1_0_3_20161220215943341.jpg.chip.jpg
      24_1_4_20170117194842219.jpg.chip.jpg
      28_0_1_20170113151917402.jpg.chip.jpg
      36_0_0_20170117181851709.jpg.chip.jpg
      27_1_0_20170117142744825.jpg.chip.jpg
      ... and 23703 more files
    crop_part1/
      1_0_3_20161220215943341.jpg.chip.jpg
      81_1_2_20170105174804349.jpg.chip.jpg
      5_1_4_20161219185942044.jpg.chip.jpg
      26_0_1_20170105183906447.jpg.chip.jpg
      43_0_0_20170104205227123.jpg.chip.jpg
      ... and 9775 more files
  crop_part1/
    1_0_3_20161220215943341.jpg.chip.jpg
    81_1

In [ ]:
# Verify dataset is properly loaded
import os

data_dir = '/content/UTKFace_data'

# Find the actual directory containing images
for root, dirs, files in os.walk(data_dir):
    image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if len(image_files) > 0:
        print(f"Found {len(image_files)} images in: {root}")
        data_dir = root  # Update data_dir to the correct path
        break

print(f"\nDataset directory: {data_dir}")
print(f"Sample filenames:")
for i, filename in enumerate(os.listdir(data_dir)[:5]):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        print(f"  {filename}")

Found 23708 images in: /content/UTKFace_data/UTKFace

Dataset directory: /content/UTKFace_data/UTKFace
Sample filenames:
  1_0_3_20161220215943341.jpg.chip.jpg
  24_1_4_20170117194842219.jpg.chip.jpg
  28_0_1_20170113151917402.jpg.chip.jpg
  36_0_0_20170117181851709.jpg.chip.jpg
  27_1_0_20170117142744825.jpg.chip.jpg


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

class UTKFaceDataset(Dataset):
    def __init__(self, data_dir, transform=None, split='train'):
        self.data_dir = data_dir
        self.transform = transform
        self.split = split

        # Load image paths and extract labels from filenames
        self.image_paths = []
        self.ages = []

        # UTK filename format: [age]_[gender]_[race]_[date&time].jpg
        for filename in os.listdir(data_dir):
            if filename.endswith('.jpg') or filename.endswith('.jpeg'):
                try:
                    age = int(filename.split('_')[0])
                    if 0 <= age <= 116:  # Valid age range
                        self.image_paths.append(os.path.join(data_dir, filename))
                        self.ages.append(age)
                except (ValueError, IndexError):
                    continue

        # Split data
        total_samples = len(self.image_paths)
        indices = np.random.RandomState(42).permutation(total_samples)

        if split == 'train':
            indices = indices[:int(0.7 * total_samples)]
        elif split == 'val':
            indices = indices[int(0.7 * total_samples):int(0.85 * total_samples)]
        else:  # test
            indices = indices[int(0.85 * total_samples):]

        self.image_paths = [self.image_paths[i] for i in indices]
        self.ages = [self.ages[i] for i in indices]

        print(f"{split} dataset size: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')

        # Get age label
        age = self.ages[idx]

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(age, dtype=torch.float32)

def get_data_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return train_transform, val_transform

def create_data_loaders(data_dir, batch_size=32):
    train_transform, val_transform = get_data_transforms()

    # Create datasets
    train_dataset = UTKFaceDataset(data_dir, transform=train_transform, split='train')
    val_dataset = UTKFaceDataset(data_dir, transform=val_transform, split='val')
    test_dataset = UTKFaceDataset(data_dir, transform=val_transform, split='test')

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader

print("Dataset classes defined successfully!")

Dataset classes defined successfully!


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class AgeDetectionModel(nn.Module):
    def __init__(self, pretrained=True, model_name='resnet50'):
        """
        Age Detection Model based on pre-trained CNN
        """
        super(AgeDetectionModel, self).__init__()

        # Load pre-trained model
        if model_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=pretrained)
            num_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()  # Remove final classification layer
        elif model_name == 'resnet34':
            self.backbone = models.resnet34(pretrained=pretrained)
            num_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
        elif model_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(pretrained=pretrained)
            num_features = self.backbone.classifier[1].in_features
            self.backbone.classifier = nn.Identity()
        else:
            raise ValueError(f"Unsupported model: {model_name}")

        # Age regression head
        self.age_regressor = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, 1)  # Single output for age
        )

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize the weights of the regression head"""
        for m in self.age_regressor.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Extract features using backbone
        features = self.backbone(x)

        # Flatten if needed
        if len(features.shape) > 2:
            features = features.view(features.size(0), -1)

        # Predict age
        age = self.age_regressor(features)

        return age.squeeze()  # Remove extra dimension

class FocalMSELoss(nn.Module):
    """
    Focal MSE Loss for age regression to focus on harder examples
    """
    def __init__(self, alpha=1.0, gamma=2.0):
        super(FocalMSELoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, predictions, targets):
        mse_loss = F.mse_loss(predictions, targets, reduction='none')
        focal_weight = self.alpha * (mse_loss ** (self.gamma / 2))
        focal_mse = focal_weight * mse_loss
        return focal_mse.mean()

def freeze_backbone(model, freeze=True):
    """
    Freeze or unfreeze the backbone parameters
    """
    for param in model.backbone.parameters():
        param.requires_grad = not freeze

    print(f"Backbone parameters {'frozen' if freeze else 'unfrozen'}")

print("✅ Model architecture defined successfully!")

✅ Model architecture defined successfully!


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(predictions, targets):
    """Calculate evaluation metrics for age prediction"""
    mae = mean_absolute_error(targets, predictions)
    mse = mean_squared_error(targets, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(targets, targets)

    # Age group accuracy (within 5 years)
    accuracy_5 = np.mean(np.abs(predictions - targets) <= 5) * 100

    # Age group accuracy (within 10 years)
    accuracy_10 = np.mean(np.abs(predictions - targets) <= 10) * 100

    return {
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'accuracy_5': accuracy_5,
        'accuracy_10': accuracy_10
    }

def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train the model for one epoch"""
    model.train()
    total_loss = 0.0
    predictions = []
    targets = []

    pbar = tqdm(train_loader, desc="Training")
    for batch_idx, (images, ages) in enumerate(pbar):
        images, ages = images.to(device), ages.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, ages)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Statistics
        total_loss += loss.item()
        predictions.extend(outputs.detach().cpu().numpy())
        targets.extend(ages.detach().cpu().numpy())

        # Update progress bar
        pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(train_loader)
    metrics = calculate_metrics(predictions, targets)

    return avg_loss, metrics

def validate_epoch(model, val_loader, criterion, device):
    """Validate the model for one epoch"""
    model.eval()
    total_loss = 0.0
    predictions = []
    targets = []

    with torch.no_grad():
        pbar = tqdm(val_loader, desc="Validation")
        for images, ages in pbar:
            images, ages = images.to(device), ages.to(device)

            outputs = model(images)
            loss = criterion(outputs, ages)

            total_loss += loss.item()
            predictions.extend(outputs.cpu().numpy())
            targets.extend(ages.cpu().numpy())

            pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(val_loader)
    metrics = calculate_metrics(predictions, targets)

    return avg_loss, metrics

def plot_training_curves(train_losses, val_losses, train_maes, val_maes):
    """Plot training curves"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss curves
    ax1.plot(train_losses, label='Train Loss', color='blue')
    ax1.plot(val_losses, label='Validation Loss', color='red')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)

    # MAE curves
    ax2.plot(train_maes, label='Train MAE', color='blue')
    ax2.plot(val_maes, label='Validation MAE', color='red')
    ax2.set_title('Training and Validation MAE')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Mean Absolute Error (years)')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

print("✅ Training utilities defined successfully!")

✅ Training utilities defined successfully!


In [ ]:
# Set up data loaders
BATCH_SIZE = 32
print(f"Creating data loaders from: {data_dir}")

try:
    train_loader, val_loader, test_loader = create_data_loaders(data_dir, BATCH_SIZE)
    print("✅ Data loaders created successfully!")

    # Show a sample batch
    for images, ages in train_loader:
        print(f"Batch shape: {images.shape}")
        print(f"Age range in batch: {ages.min():.1f} - {ages.max():.1f} years")
        break

except Exception as e:
    print(f"❌ Error creating data loaders: {e}")
    print("Please check your dataset path and format")

Creating data loaders from: /content/UTKFace_data/UTKFace
train dataset size: 16595
val dataset size: 3556
test dataset size: 3557
✅ Data loaders created successfully!
Batch shape: torch.Size([32, 3, 224, 224])
Age range in batch: 1.0 - 84.0 years


In [ ]:
# Training configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_NAME = 'resnet50'  # Options: 'resnet50', 'resnet34', 'efficientnet_b0'
EPOCHS = 20  # Start with fewer epochs for testing
LEARNING_RATE = 1e-4
USE_FOCAL_LOSS = False  # Set to True to use Focal MSE Loss

print(f"🖥️ Device: {DEVICE}")
print(f"🧠 Model: {MODEL_NAME}")
print(f"⏱️ Epochs: {EPOCHS}")
print(f"📈 Learning Rate: {LEARNING_RATE}")
print(f"🎯 Use Focal Loss: {USE_FOCAL_LOSS}")

# Create model
model = AgeDetectionModel(pretrained=True, model_name=MODEL_NAME)
model = model.to(DEVICE)

# Initially freeze backbone
freeze_backbone(model, freeze=True)

# Loss function
if USE_FOCAL_LOSS:
    criterion = FocalMSELoss(alpha=1.0, gamma=2.0)
    print("Using Focal MSE Loss")
else:
    criterion = nn.MSELoss()
    print("Using MSE Loss")

# Optimizer and scheduler
backbone_params = list(model.backbone.parameters())
head_params = list(model.age_regressor.parameters())

optimizer = torch.optim.AdamW([
    {'params': backbone_params, 'lr': LEARNING_RATE * 0.1},
    {'params': head_params, 'lr': LEARNING_RATE}
], weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

print("\n✅ Model and training setup complete!")
print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

🖥️ Device: cpu
🧠 Model: resnet50
⏱️ Epochs: 20
📈 Learning Rate: 0.0001
🎯 Use Focal Loss: False


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Backbone parameters frozen
Using MSE Loss

✅ Model and training setup complete!
📊 Model parameters: 1,180,673


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# COMPLETE FIXED TRAINING CELL
print("🚀 Starting Training with Fixed Metrics...")
print("=" * 60)

# Initialize tracking variables
best_val_mae = float('inf')
train_losses = []
val_losses = []
train_maes = []
val_maes = []

# Fixed training function
def train_epoch_fixed(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    all_predictions = []
    all_targets = []

    pbar = tqdm(train_loader, desc="Training")
    for batch_idx, (images, ages) in enumerate(pbar):
        images, ages = images.to(device), ages.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, ages)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Convert to numpy immediately
        batch_preds = outputs.detach().cpu().numpy().flatten()
        batch_targets = ages.detach().cpu().numpy().flatten()

        all_predictions.extend(batch_preds)
        all_targets.extend(batch_targets)

        pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(train_loader)

    # Calculate metrics with numpy arrays
    predictions = np.array(all_predictions)
    targets = np.array(all_targets)

    mae = np.mean(np.abs(predictions - targets))
    accuracy_5 = np.mean(np.abs(predictions - targets) <= 5) * 100
    accuracy_10 = np.mean(np.abs(predictions - targets) <= 10) * 100

    metrics = {
        'mae': mae,
        'accuracy_5': accuracy_5,
        'accuracy_10': accuracy_10
    }

    return avg_loss, metrics

def validate_epoch_fixed(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        pbar = tqdm(val_loader, desc="Validation")
        for images, ages in pbar:
            images, ages = images.to(device), ages.to(device)

            outputs = model(images)
            loss = criterion(outputs, ages)

            total_loss += loss.item()

            # Convert to numpy immediately
            batch_preds = outputs.cpu().numpy().flatten()
            batch_targets = ages.cpu().numpy().flatten()

            all_predictions.extend(batch_preds)
            all_targets.extend(batch_targets)

            pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(val_loader)

    # Calculate metrics with numpy arrays
    predictions = np.array(all_predictions)
    targets = np.array(all_targets)

    mae = np.mean(np.abs(predictions - targets))
    accuracy_5 = np.mean(np.abs(predictions - targets) <= 5) * 100
    accuracy_10 = np.mean(np.abs(predictions - targets) <= 10) * 100

    metrics = {
        'mae': mae,
        'accuracy_5': accuracy_5,
        'accuracy_10': accuracy_10
    }

    return avg_loss, metrics

# Phase 1: Train with frozen backbone
phase1_epochs = min(6, EPOCHS // 3)
print(f"\n📘 PHASE 1: Training with frozen backbone ({phase1_epochs} epochs)")
print("-" * 50)

for epoch in range(phase1_epochs):
    print(f"\n🔄 Epoch {epoch+1}/{phase1_epochs}")

    # Training
    train_loss, train_metrics = train_epoch_fixed(model, train_loader, criterion, optimizer, DEVICE)

    # Validation
    val_loss, val_metrics = validate_epoch_fixed(model, val_loader, criterion, DEVICE)

    # Learning rate scheduling
    scheduler.step(val_loss)

    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_maes.append(train_metrics['mae'])
    val_maes.append(val_metrics['mae'])

    # Print results
    print(f"📊 Train Loss: {train_loss:.4f}, Train MAE: {train_metrics['mae']:.2f}")
    print(f"📊 Val Loss: {val_loss:.4f}, Val MAE: {val_metrics['mae']:.2f}")
    print(f"🎯 Accuracy (±5 years): {val_metrics['accuracy_5']:.1f}%")

    # Save best model
    if val_metrics['mae'] < best_val_mae:
        best_val_mae = val_metrics['mae']
        torch.save(model.state_dict(), '/content/best_model.pth')
        print(f"💾 New best model saved! MAE: {best_val_mae:.2f}")

print(f"\n✅ Phase 1 completed! Best MAE: {best_val_mae:.2f} years")

In [ ]:
# Phase 2: Fine-tune with unfrozen backbone
print(f"\n🔥 PHASE 2: Fine-tuning with unfrozen backbone")
print("-" * 50)

# Unfreeze backbone
freeze_backbone(model, freeze=False)

# Create new optimizer with lower learning rate
optimizer = torch.optim.AdamW([
    {'params': backbone_params, 'lr': LEARNING_RATE * 0.01},  # Very low LR for backbone
    {'params': head_params, 'lr': LEARNING_RATE * 0.1}  # Lower LR for head
], weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

phase2_epochs = EPOCHS - phase1_epochs

for epoch in range(phase2_epochs):
    print(f"\n🔄 Epoch {epoch+1}/{phase2_epochs} (Phase 2)")

    # Training
    train_loss, train_metrics = train_epoch_fixed(model, train_loader, criterion, optimizer, DEVICE)

    # Validation
    val_loss, val_metrics = validate_epoch_fixed(model, val_loader, criterion, DEVICE)

    # Learning rate scheduling
    scheduler.step(val_loss)

    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_maes.append(train_metrics['mae'])
    val_maes.append(val_metrics['mae'])

    # Print results
    print(f"📊 Train Loss: {train_loss:.4f}, Train MAE: {train_metrics['mae']:.2f}")
    print(f"📊 Val Loss: {val_loss:.4f}, Val MAE: {val_metrics['mae']:.2f}")
    print(f"🎯 Accuracy (±5 years): {val_metrics['accuracy_5']:.1f}%")

    # Save best model
    if val_metrics['mae'] < best_val_mae:
        best_val_mae = val_metrics['mae']
        torch.save(model.state_dict(), '/content/best_model.pth')
        print(f"💾 New best model saved! MAE: {best_val_mae:.2f}")

print(f"\n🎉 Training completed! Best validation MAE: {best_val_mae:.2f} years")

In [ ]:
# Plot training curves
plt.figure(figsize=(15, 5))

# Loss curves
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue', linewidth=2)
plt.plot(val_losses, label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# MAE curves
plt.subplot(1, 2, 2)
plt.plot(train_maes, label='Train MAE', color='blue', linewidth=2)
plt.plot(val_maes, label='Validation MAE', color='red', linewidth=2)
plt.title('Training and Validation MAE', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error (years)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📈 Training Summary:")
print(f"🔹 Total Epochs: {len(train_losses)}")
print(f"🔹 Best Validation MAE: {best_val_mae:.2f} years")
print(f"🔹 Final Train MAE: {train_maes[-1]:.2f} years")
print(f"🔹 Final Val MAE: {val_maes[-1]:.2f} years")

In [ ]:
# Load best model and evaluate on test set
print("📂 Loading best model for final evaluation...")
model.load_state_dict(torch.load('/content/best_model.pth'))

# Evaluate on test set
print("🧪 Evaluating on test set...")
model.eval()
all_predictions = []
all_targets = []

with torch.no_grad():
    for images, ages in tqdm(test_loader, desc="Testing"):
        images, ages = images.to(DEVICE), ages.to(DEVICE)
        outputs = model(images)

        # Convert to numpy immediately
        batch_preds = outputs.cpu().numpy().flatten()
        batch_targets = ages.cpu().numpy().flatten()

        all_predictions.extend(batch_preds)
        all_targets.extend(batch_targets)

# Convert to numpy arrays
predictions = np.array(all_predictions)
targets = np.array(all_targets)

# Calculate comprehensive metrics
mae = np.mean(np.abs(predictions - targets))
mse = np.mean((predictions - targets) ** 2)
rmse = np.sqrt(mse)
accuracy_5 = np.mean(np.abs(predictions - targets) <= 5) * 100
accuracy_10 = np.mean(np.abs(predictions - targets) <= 10) * 100

# Calculate R² score
from sklearn.metrics import r2_score
r2 = r2_score(targets, predictions)

print("\n" + "="*60)
print("🎯 FINAL TEST RESULTS")
print("="*60)
print(f"📊 Total test samples: {len(predictions)}")
print(f"📏 Mean Absolute Error (MAE): {mae:.2f} years")
print(f"📐 Root Mean Square Error (RMSE): {rmse:.2f} years")
print(f"📈 R² Score: {r2:.4f}")
print(f"🎯 Accuracy (±5 years): {accuracy_5:.1f}%")
print(f"🎯 Accuracy (±10 years): {accuracy_10:.1f}%")
print("="*60)

# Store results for later use
final_metrics = {
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'accuracy_5': accuracy_5,
    'accuracy_10': accuracy_10,
    'predictions': predictions,
    'targets': targets
}

In [ ]:
# Create comprehensive evaluation visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Scatter plot: Predictions vs True Ages
ax1.scatter(targets, predictions, alpha=0.6, s=20, color='blue')
ax1.plot([0, 100], [0, 100], 'r--', linewidth=3, label='Perfect Prediction')
ax1.set_xlabel('True Age', fontsize=12)
ax1.set_ylabel('Predicted Age', fontsize=12)
ax1.set_title(f'Predictions vs True Ages\n(MAE: {mae:.2f} years)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Error histogram
errors = predictions - targets
ax2.hist(errors, bins=50, alpha=0.7, edgecolor='black', color='orange')
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2, alpha=0.8)
ax2.set_xlabel('Prediction Error (years)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Distribution of Prediction Errors', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Age distribution comparison
ax3.hist(targets, bins=30, alpha=0.7, label='True Ages', edgecolor='black', color='green')
ax3.hist(predictions, bins=30, alpha=0.7, label='Predicted Ages', edgecolor='black', color='purple')
ax3.set_xlabel('Age', fontsize=12)
ax3.set_ylabel('Frequency', fontsize=12)
ax3.set_title('Age Distribution Comparison', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Accuracy by age group
age_bins = np.arange(0, 101, 10)
accuracies = []
bin_centers = []

for i in range(len(age_bins) - 1):
    mask = (targets >= age_bins[i]) & (targets < age_bins[i+1])
    if mask.sum() > 5:  # Only consider bins with enough samples
        acc = np.mean(np.abs(predictions[mask] - targets[mask]) <= 5) * 100
        accuracies.append(acc)
        bin_centers.append((age_bins[i] + age_bins[i+1]) / 2)

ax4.bar(bin_centers, accuracies, width=8, alpha=0.7, edgecolor='black', color='red')
ax4.set_xlabel('Age Group (center)', fontsize=12)
ax4.set_ylabel('Accuracy within ±5 years (%)', fontsize=12)
ax4.set_title('Accuracy by Age Group', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Age group detailed analysis
print("\n📈 Age Group Performance Analysis:")
print("-" * 60)
age_groups = [(0, 18), (18, 30), (30, 50), (50, 70), (70, 120)]

for min_age, max_age in age_groups:
    mask = (targets >= min_age) & (targets < max_age)
    if mask.sum() > 0:
        group_mae = np.mean(np.abs(predictions[mask] - targets[mask]))
        group_acc5 = np.mean(np.abs(predictions[mask] - targets[mask]) <= 5) * 100
        group_count = mask.sum()
        print(f"Age {min_age:2d}-{max_age:2d}: MAE={group_mae:5.2f} years, Acc(±5)={group_acc5:5.1f}%, n={group_count:3d}")

In [ ]:
# Save and download the trained model
from google.colab import files
import json

# Save comprehensive results
results = {
    'final_metrics': {
        'mae': float(mae),
        'rmse': float(rmse),
        'r2': float(r2),
        'accuracy_5': float(accuracy_5),
        'accuracy_10': float(accuracy_10)
    },
    'training_history': {
        'train_losses': [float(x) for x in train_losses],
        'val_losses': [float(x) for x in val_losses],
        'train_maes': [float(x) for x in train_maes],
        'val_maes': [float(x) for x in val_maes]
    },
    'model_config': {
        'model_name': MODEL_NAME,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'use_focal_loss': USE_FOCAL_LOSS,
        'best_val_mae': float(best_val_mae)
    }
}

# Save model state and results
torch.save({
    'model_state_dict': model.state_dict(),
    'results': results
}, '/content/age_detection_model_complete.pth')

# Save results as JSON for easy reading
with open('/content/training_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("💾 Files ready for download:")
print("- age_detection_model_complete.pth: Complete model with training history")
print("- training_results.json: Training results in readable format")

# Download the files
files.download('/content/age_detection_model_complete.pth')
files.download('/content/training_results.json')

print(f"\n🎉 Age Detection Training Completed Successfully!")
print(f"📊 Final MAE: {mae:.2f} years")
print(f"🎯 Accuracy (±5 years): {accuracy_5:.1f}%")
print(f"🎯 Accuracy (±10 years): {accuracy_10:.1f}%")
print(f"🏆 Model Performance Grade: {'Excellent' if mae < 4 else 'Very Good' if mae < 6 else 'Good' if mae < 8 else 'Fair'}")